# HAVE A LAN? USE IT THROUGH HDDM !

If you have you own LAN, or really, a class with a `predict_on_batch()` method which you can call to get back the log-likelihood of a dataset, 

you can pass it as an argument to the `HDDMnn`, `HDDMnnRegressor` or `HDDMnnStimCoding` classes. 
However in addition we also need to construct a likelihood function that PyMC2 can accept. To help with this we provide convenience functions, which make this process simple.

In this document we provide you with a simple complete example.

Let's say we have a new model, the `test_model`, and trained a LAN for it. 
In the example we are going to use a `Keras` model for the LAN, however as mentioned above, a class with `predict_on_batch()` method will work too.

We will have to go through two preparatory steps, before we can pass our model to one of the `HDDMnn..()` classes.

1. Construct a config dictionary for our `test_model`
2. Construct a PyMC2 ready likelihood function for the `test_model`

We will put a runnable example below, however keep the main part of this document non-executable so you do not need an `HDDM` ready environment to read the basic instructions in this tutorial.

### CONSTRUCT THE CONFIG DICTIONARY

You will first have to load the HDDM package into your environment,

```python
import hddm
```

you can then access the config dictionary for all models included in `HDDM` by calling the  `hddm.model_config` object.

You want to add your `test_model` metadata to the model_config. (We use the config form the `ddm` model for our `test_model` here)

```python
hddm.model_config["test_model"]: {
        "params": ["v", "a", "z", "t"], # parameter names associated to your model 
        "params_trans": [0, 0, 1, 0], # for each parameter do you want the sampler to transform it into an unconstrained space? (invlogit <--> logistic)
        "params_std_upper": [1.5, 1.0, None, 1.0], # for group models, what is the maximal standard deviation to consider for the prior on the parameter
        "param_bounds": [[-3.0, 0.3, 0.1, 1e-3], [3.0, 2.5, 0.9, 2.0]], # the parameter boundaries you used for training your LAN
        "boundary": hddm.simulators.bf.constant, # add a boundary function (if relevant to your model) (optional)
        "n_params": 4, # number of parameters of your model
        "default_params": [0.0, 1.0, 0.5, 1e-3], # defaults for each parameter 
        "hddm_include": ["z"], # suggestion for which parameters to include via the include statement of an HDDM model (usually you want all of the parameters from above)
        "n_choices": 2, # number of choice options of the model
        "choices": [-1, 1], # choice labels (what your simulator spits out)
        "slice_widths": {"v": 1.5, "v_std": 1,  
                         "a": 1, "a_std": 1, 
                         "z": 0.1, "z_trans": 0.2, 
                         "t": 0.01, "t_std": 0.15}, # hyperparameters for the slice-sampler used for posterior sampling, take these as an orientation, can be helpful to optimize speed (optional)
    }
```

### CONSTRUCT PyMC2 READY LIKELIHOOD

Once you have you config dictionary set up, you can now construct a PyMC2 ready likelihood, by using the folowing convenience functions:

```python
likelihood_string = hddm.utils.make_likelihood_str_mlp(config = hddm.simulators.model_config['test_model'])
```

The `hddm.utils.make_likelihood_str_mlp()` constructs a string which holds the code to define a PyMC2 ready likelihood function from your `test_model` config dictionary.

Now you can call the, `hddm.utils.make_likelihood_fun_from_str()` function, to actually define the likelihood function:

```python
hddm.utils.make_likelihood_fun_from_str(likelihood_string)
```

You now have a function called `custom_likelihood` available in your environment.

### LOAD THE NETWORK

To make the example complete, here is a code snippet to load in a `Keras` model.

```python
my_network = keras.models.load_model('model_final_ddm.h5', compile = False) # or any class with a valid predict on batch function
```

### INITIALIZE THE HDDM MODEL

Now you are ready to load a HDDM model with your `test_model` LAN.

```python
# Define the HDDM model
hddmnn_model = hddm.HDDMnn(data = some_dataset,
                           informative = False,
                           include = hddm.model_config['test_model']['hddm_include'], # Note: This include statement is an example, you may pick any other subset of the parameters of your model here
                           model = 'test_model',
                           network = my_network,
                           custom_likelihood = custom_likelihood)
```


You are now ready to get samples from your model.

### WARNING

Not all the functionality of the HDDM package will work seamlessly with such custom likelihoods. 
You will be able to generate some, but not all plots.

The utility lies in using HDDM as a vehicle to sample from user defined approximate likelihoods.
Most of the packages utility functions have a higher degree of specificity to models that have been fully incorporated into the package.

Look at the tutorial in tutorial `add_new_models_to_hddm_tutorial.ipynb` for a higher degree of integration.

### END